In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score

In [5]:
rides= pd.read_csv(r"E:\Projects\Machine Learning Projects\Uber Price Surge Prediction\cab_rides.csv", delimiter=',')
weather= pd.read_csv(r"E:\Projects\Machine Learning Projects\Uber Price Surge Prediction\weather (1).csv", delimiter=",")

In [6]:
rides.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL


In [7]:
weather.head()

,temp,location,clouds,pressure,rain,time_stamp,humidity,wind
0,42.42,Back Bay,1.0,1012.14,0.1228,1545003901,0.77,11.25
1,42.43,Beacon Hill,1.0,1012.15,0.1846,1545003901,0.76,11.32
2,42.50,Boston University,1.0,1012.15,0.1089,1545003901,0.76,11.07
3,42.11,Fenway,1.0,1012.13,0.0969,1545003901,0.77,11.09
4,43.13,Financial District,1.0,1012.14,0.1786,1545003901,0.75,11.49


In [8]:
rides['date_time'] = pd.to_datetime(rides['time_stamp']/1000, unit='s')
weather['date_time'] = pd.to_datetime(weather['time_stamp'], unit='s')
rides.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,date_time
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,2018-12-16 09:30:07.890000128
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,2018-11-27 02:00:23.676999936
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft,2018-11-28 01:00:22.197999872
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL,2018-11-30 04:53:02.749000192
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL,2018-11-29 03:49:20.223000064


In [9]:
rides['merge_date'] = rides.source.astype(str) +" - "+ rides.date_time.dt.date.astype("str") +" - "+ rides.date_time.dt.hour.astype("str")
weather['merge_date'] = weather.location.astype(str) +" - "+ weather.date_time.dt.date.astype("str") +" - "+ weather.date_time.dt.hour.astype("str")

In [7]:
#final_dataframe=final_dataframe.dropna(axis=0)

In [10]:
weather.index = weather['merge_date']

final_dataframe = rides.join(weather,on=['merge_date'],rsuffix ='_w')

In [12]:
final_dataframe=final_dataframe.dropna(axis=0)

In [13]:
final_dataframe['day'] = final_dataframe.date_time.dt.dayofweek
final_dataframe['hour'] = final_dataframe.date_time.dt.hour

In [14]:
surge_dataframe = final_dataframe[final_dataframe.surge_multiplier < 3]

In [15]:
x = surge_dataframe[['distance','day','hour','temp','clouds', 'pressure','humidity', 'wind', 'rain']]

y=surge_dataframe['surge_multiplier']

le = LabelEncoder()

In [16]:
le.fit([1,1.25,1.5,1.75,2.,2.25,2.5])
y = le.transform(y)

In [17]:
feature_list=list(x.columns)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)


In [18]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
x_train, y_train = sm.fit_resample(x_train, y_train)

In [ ]:
#pip install imblearn --user

In [19]:
model= RandomForestClassifier(n_jobs=-1, random_state = 42,class_weight="balanced")

model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [20]:
print(pd.crosstab(le.inverse_transform(y_test), le.inverse_transform(y_pred),rownames=['Actual'],colnames=['Predicted']))

Predicted   1.00  1.25  1.50  1.75  2.00  2.50
Actual                                        
1.00       53719   828   432   165   144     7
1.25         366   581    10     3     7     0
1.50         194     4   200     7     2     0
1.75         101     2     1   129     1     0
2.00          65     7     3     4    99     2
2.50           4     0     0     0     4     1


In [21]:
print("Accuracy Score = ",accuracy_score(y_test,y_pred))              
print("precision score = ",precision_score(y_test, y_pred,average='weighted'))         
print("recall score = ",recall_score(y_test, y_pred,average='micro'))

Accuracy Score =  0.9586106634905065
precision score =  0.9676137671296904
recall score =  0.9586106634905065


In [22]:
from sklearn.metrics import f1_score
print("f1 score = ",f1_score(y_test, y_pred, average='weighted'))
errors = abs(y_pred - y_test)

f1 score =  0.9625158233169181


In [23]:
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.07 degrees.


In [24]:
importances = list(model.feature_importances_)

In [25]:
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

In [26]:
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

In [27]:
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: distance             Importance: 0.31
Variable: temp                 Importance: 0.13
Variable: pressure             Importance: 0.13
Variable: wind                 Importance: 0.13
Variable: rain                 Importance: 0.11
Variable: humidity             Importance: 0.1
Variable: hour                 Importance: 0.05
Variable: clouds               Importance: 0.04
Variable: day                  Importance: 0.02


[None, None, None, None, None, None, None, None, None]